# Dependencies

In [163]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

# Creating Training Data

In [164]:
training_df = pd.read_csv('training_data.csv', index_col=0)
# training_df = pd.read_csv('training_data_v2.csv', index_col=0)

training_df['Analyst Rating'] = ""

buy_classification = ['buy', 'outperform', 'positive']

for index, row in training_df.iterrows():
    ticker = row['Ticker']
    stock_df = pd.read_csv('../Data_Collection/Analyst_Ratings/' + ticker + '.csv')
    classification_dict = {"Buy": 0, "Don't Buy": 0}
    
    for item in stock_df['Rating']:
        intersection = list(set(str(item).lower().split(' ')) & set(buy_classification))
        if len(intersection) > 0:
            classification_dict["Buy"] += 1
        else:
            classification_dict["Don't Buy"] += 1
            
    values = list(classification_dict.values())
    keys = list(classification_dict.keys())
    if keys[values.index(max(values))] == "Buy":
        training_df.at[index, 'Analyst Rating'] = 1
    else:
        training_df.at[index, 'Analyst Rating'] = 0

# Scaling the data
Scaler = MinMaxScaler()
cols = [x for x in training_df.columns if x not in ['Analyst Rating', 'Ticker']]
training_df[cols] = Scaler.fit_transform(training_df[cols])
training_df = training_df.dropna(axis=0)

training_df

,Ticker,Conversations,Headlines,payoutRatio,beta,regularMarketVolume,profitMargins,52WeekChange,forwardEps,bookValue,...,heldPercentInsiders,mostRecentQuarter,nextFiscalYearEnd,shortRatio,enterpriseValue,earningsQuarterlyGrowth,sharesShortPriorMonth,shortPercentOfFloat,pegRatio,Analyst Rating
0,NVDA,0.424561,0.892070,0.128478,0.591948,0.052327,0.943654,0.184837,0.178523,0.071583,...,0.208158,0.562500,1.000000,0.077273,0.139727,0.421810,0.034314,0.079612,0.007925,1
1,MSFT,0.729825,0.378855,0.431261,0.214261,0.258086,0.996943,0.090959,0.093979,0.040912,...,0.001503,0.078125,0.122449,0.386364,0.841075,0.365071,0.376497,0.000000,0.010038,1
2,TSLA,0.533333,0.555066,0.000000,1.000000,0.331638,0.775201,1.000000,0.062197,0.059005,...,1.000000,0.078125,0.873469,0.490909,0.296855,0.709383,0.447375,1.000000,0.028119,1
3,AAPL,0.800000,0.665198,0.301398,0.490618,1.000000,0.913480,0.163087,0.051769,0.000000,...,0.001804,0.000000,0.481633,0.209091,1.000000,0.355660,0.804040,0.009709,0.009862,1
4,GOOGL,0.445614,0.389868,0.000000,0.331969,0.000000,0.915805,0.135370,1.000000,1.000000,...,0.004660,0.078125,0.873469,0.490909,0.606609,0.392748,0.000000,0.091262,0.009099,1
5,FB,0.287719,0.387665,0.000000,0.491982,0.241268,1.000000,0.136918,0.161763,0.126191,...,0.030216,0.078125,0.873469,0.363636,0.365047,0.420426,0.184465,0.095146,0.005753,1
6,JNJ,0.473684,0.801762,1.000000,0.150461,0.042018,0.885624,0.052244,0.121167,0.061726,...,0.002606,0.125000,0.885714,0.536364,0.193772,0.117631,0.115055,0.005825,0.016730,1
7,VZ,0.385965,0.563877,0.800083,0.000000,0.172179,0.857597,0.012089,0.058101,0.038256,...,0.000000,0.078125,0.873469,0.754545,0.159684,0.246886,0.352190,0.114563,0.019313,0
10,CSCO,0.031579,0.325991,0.834141,0.275674,0.246174,0.908902,0.038614,0.036002,0.016373,...,0.001503,0.437500,0.224490,0.709091,0.074272,0.242458,0.405601,0.095146,0.012562,1
11,PYPL,0.694737,1.000000,0.000000,0.427158,0.031599,0.898201,0.261882,0.064929,0.040365,...,0.003808,0.078125,0.873469,0.263636,0.117848,0.853308,0.116768,0.106796,0.012856,1


# Developing the model

In [165]:
LR = LogisticRegression(penalty='l2')

training_cols = [x for x in training_df.columns if x not in ['Analyst Rating', 'Ticker']]
X_train, X_test, y_train, y_test = train_test_split(training_df[training_cols], 
                                                    training_df['Analyst Rating'], test_size=0.27, random_state=42)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

LR = LR.fit(X_train,y_train)
y_predictions = LR.predict(X_test)

print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

